In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import Ridge

from sklearn.impute import SimpleImputer

import sys
sys.path.append('../../modules')
from helper_functions import feature_reduction_x

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/jbyrneirl/BER-Rating-AI-Model/main/data/training/BERRatingData_aa.csv', sep=';', on_bad_lines="skip", low_memory=False)

np.random.seed(0)

#X = df.drop("BerRating", axis='columns')
#X = pd.get_dummies(X)
X = feature_reduction_x(df)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X)
X = imp.transform(X)
y = df.BerRating


X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=2)

c:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [3]:
model = Ridge()


model.fit(X_train , y_train)
score = model.score(X_test , y_test)

In [4]:
print(score)

0.9892906126425163
